In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # activation funcion 사용을 위해 import
import torchvision
from torchvision import datasets, transforms # cifar10 데이터 가져오기 위해서 사용.
     


In [9]:
BATCH_SIZE = 32
train_dataset = datasets.CIFAR10(root="./data/",
                                 train=True,
                                 download=True,
                                 transform=transforms.ToTensor())

test_dataset = datasets.CIFAR10(root="./data/",
                                train=False,
                                download=True,
                                transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)

print(train_loader.dataset)
# Dataset CIFAR10
#    Number of datapoints: 50000
#    Root location: ./data/
#    Split: Train
#    StandardTransform
#    Transform: ToTensor()

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [10]:
for (X_train, Y_train) in train_loader:
    print(f"X_train: {X_train.size()} type: {X_train.type()}")
    print(f"Y_train: {Y_train.size()} type: {Y_train.type()}")
    break
# X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
# Y_train: torch.Size([32]) type: torch.LongTensor

X_train: torch.Size([32, 3, 32, 32]) type: torch.FloatTensor
Y_train: torch.Size([32]) type: torch.LongTensor


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=8,
            kernel_size=3,
            padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            padding=1)
        self.pool = nn.MaxPool2d(
            kernel_size=2,
            stride=2
        )
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool(x)

        x = x.view(-1, 8 * 8 * 16)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.log_softmax(x, dim=1)
        return x

In [12]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

print(f"Using PyTorch version: {torch.__version__}, Device: {DEVICE}")

model = CNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

Using PyTorch version: 2.0.0+cu118, Device: cpu


In [13]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print(
                f"train Epoch: {Epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)}({100. * batch_idx / len(train_loader):.0f}%)]\tTrain Loss: {loss.item()}")

In [14]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [16]:
EPOCHS = 100
for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval=200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print(f"\n[EPOCH: {Epoch}]\tTest Loss: {test_loss:.4f}\tTest Accuracy: {test_accuracy} % \n")

train Epoch: 1 [0/50000(0%)]	Train Loss: 0.7328563928604126
train Epoch: 1 [6400/50000(13%)]	Train Loss: 0.8614412546157837
train Epoch: 1 [12800/50000(26%)]	Train Loss: 0.9480295777320862
train Epoch: 1 [19200/50000(38%)]	Train Loss: 0.7398285269737244
train Epoch: 1 [25600/50000(51%)]	Train Loss: 0.7575651407241821
train Epoch: 1 [32000/50000(64%)]	Train Loss: 0.8179744482040405
train Epoch: 1 [38400/50000(77%)]	Train Loss: 0.9900941848754883
train Epoch: 1 [44800/50000(90%)]	Train Loss: 0.9348811507225037

[EPOCH: 1]	Test Loss: 0.0326	Test Accuracy: 63.67 % 

train Epoch: 2 [0/50000(0%)]	Train Loss: 1.0386896133422852
train Epoch: 2 [6400/50000(13%)]	Train Loss: 0.7888926863670349
train Epoch: 2 [12800/50000(26%)]	Train Loss: 0.7222089767456055
train Epoch: 2 [19200/50000(38%)]	Train Loss: 0.5918994545936584
train Epoch: 2 [25600/50000(51%)]	Train Loss: 1.0157147645950317
train Epoch: 2 [32000/50000(64%)]	Train Loss: 0.7719913721084595
train Epoch: 2 [38400/50000(77%)]	Train Loss: 0